In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg

#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python

import cv2

In [22]:
from glob import glob
from tqdm.notebook import tqdm
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix

In [5]:
prj_fd = "/home/humble_local2_25t/alexw/MIST_round3/MIST_additional_holdout/data/"
data_names = os.listdir(prj_fd)
data_names.remove("spot_metadata.tsv")
data_names

['Human_Breast_Cancer_Ductal_Carcinoma_FFPE',
 'Human_Breast_Cancer_BAS1',
 'Human_Prostate_Cancer_Adenocarcinoma_FFPE',
 'MouseAD',
 'Human_Glioblastoma',
 'Human_Invasive_Ductal_Carcinoma',
 'Human_Prostate_Cancer_Acinar_Cell_Carcinoma_FFPE',
 'Human_Breast_Cancer_BAS2',
 'Prostate',
 'MouseWT',
 'Melanoma',
 'Human_Ovarian_Cancer',
 'Human_Colorectal_Cancer']

In [14]:
meta = pd.read_csv(f"{prj_fd}spot_metadata.tsv",sep="\t")
meta.index = meta[['coord_X', 'coord_Y']].apply(lambda x: 'x'.join(x.astype(str)), axis=1)
meta = meta[['coord_Y', 'coord_X', 'img_coord_Y', 'img_coord_X', 'Group']]
meta_ad = meta.loc[meta.Group == 'AD_12']
meta_wt = meta.loc[meta.Group == 'WT_12']

In [23]:
for i, dn in enumerate(tqdm(data_names)):
    if dn in ['Prostate', 'Melanoma']:
        continue
    else:        
        fp = f"{prj_fd}{dn}/raw.csv"
        df = pd.read_csv(fp, index_col=0)
        mist_results = pd.read_csv(f"{prj_fd}{dn}/cluster_results_all_methods.csv", index_col=0)
        df = df.loc[mist_results.index,:]
        mist_clusters = set(mist_results.MIST)
        mist_clusters.remove("isolated")
        n_clusters = len(mist_clusters)
        
        if dn == 'MouseAD':
            x_array=meta_ad.loc[df.index, "coord_X"].tolist()
            y_array=meta_ad.loc[df.index, "coord_Y"].tolist()
            x_pixel=meta_ad.loc[df.index, "img_coord_X"].tolist()
            y_pixel=meta_ad.loc[df.index, "img_coord_Y"].tolist()
        elif dn == 'MouseWT':
            x_array=meta_wt.loc[df.index, "coord_X"].tolist()
            y_array=meta_wt.loc[df.index, "coord_Y"].tolist()
            x_pixel=meta_wt.loc[df.index, "img_coord_X"].tolist()
            y_pixel=meta_wt.loc[df.index, "img_coord_Y"].tolist()
        else:
            spatial_df = pd.read_csv(f"{prj_fd}{dn}/tissue_positions_list.csv", header=None)
            spatial_df.columns = ['barcode', 'in_tissue', 'array_row', 'array_col', 'img_x', 'img_y']
            spatial_df.index = spatial_df[['array_col', 'array_row']].apply(lambda x: 'x'.join(x.astype(str)), axis=1)
            spatial_df = spatial_df.loc[df.index,:]        
            x_array = spatial_df.array_col.tolist()
            y_array = spatial_df.array_row.tolist()
            x_pixel = spatial_df.img_x.tolist()
            y_pixel = spatial_df.img_y.tolist()
        s=1
        b=49
        adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
        obs_df = pd.DataFrame({'x_array': x_array, 'y_array': y_array,
                                              'x_pixel': x_pixel, 'y_pixel': y_pixel},
                                              index = df.index)
        obs_df = pd.concat([obs_df, mist_results], axis=1)
        adata = ad.AnnData(X=csr_matrix(df.values), 
                           var = pd.DataFrame(index = df.columns),
                           obs = obs_df)
        adata.var_names_make_unique()
        spg.prefilter_genes(adata,min_cells=3) # avoiding all genes are zeros
        spg.prefilter_specialgenes(adata)
        #Normalize and take log for UMI
        sc.pp.normalize_per_cell(adata)
        sc.pp.log1p(adata)
        p=0.5 
        l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)
        #Set seed
        r_seed=t_seed=n_seed=100
        #Search for suitable resolution
        res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)
        clf=spg.SpaGCN()
        clf.set_l(l)
        #Set seed
        random.seed(r_seed)
        torch.manual_seed(t_seed)
        np.random.seed(n_seed)
        #Run
        clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
        y_pred, prob=clf.predict()
        adata.obs["pred"]= y_pred
        adata.obs["pred"]=adata.obs["pred"].astype('category')
        #Do cluster refinement(optional)
        #shape="hexagon" for Visium data, "square" for ST data.
        adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
        refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
        adata.obs["refined_pred"]=refined_pred
        adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')

        obs = adata.obs.copy()
        obs['barcode'] = obs.index
        obs.to_csv(f"{prj_fd}{dn}/cluster_results_all_methods_with_SpaGCN.csv")

  0%|          | 0/13 [00:00<?, ?it/s]

Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 6.9989223767959e+01]
Run 2: l [0.01, 500.005], p [0.0, 18.055377960205078]
Run 3: l [0.01, 250.0075], p [0.0, 3.9920783042907715]
Run 4: l [125.00874999999999, 250.0075], p [0.3727337121963501, 3.9920783042907715]
Run 5: l [125.00874999999999, 187.508125], p [0.3727337121963501, 1.8593666553497314]
Run 6: l [125.00874999999999, 156.2584375], p [0.3727337121963501, 1.0176112651824951]
Run 7: l [125.00874999999999, 140.63359375], p [0.3727337121963501, 0.6644240617752075]
recommended l =  132.821171875
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7


2022-06-06 01:39:32.057100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/humble_local_25t/alexw/miniconda3/envs/ReST/lib/python3.9/site-packages/cv2/../../lib64:
2022-06-06 01:39:32.057129: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  14
Initializing cluster centers with louvain, resolution =  0.6
Epoch  0
Epoch  10
Res =  0.6 Num of clusters =  12
Res changed to 0.6
Initializing cluster centers with louvain, resolution =  0.5
Epoch  0
Epoch  10
Res =  0.5 Num of clusters =  10
Res changed to 0.5
Initializing cluster centers with louvain, resolution =  0.4
Epoch  0
Epoch  10
Res =  0.4 Num of clusters =  9
Res changed to 0.4
Initializing cluster centers with louvain, resolution =  0.30000000000000004
Epoch  0
Epoch  10
Res =  0.30000000000000004 Num of clusters =  7
recommended res =  0.30000000000000004
Initializing cluster centers with louvain, resolution =  0.30000000000000004
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch  50
Epoch  60
Epoch  70
Epoch  80
Epoch  90
Epoch  100
Epoch  110
Epoch  120
Epoch  130
Epoch  140
Epoch  150
Epoch  160
Epoch  170
Epoch  180
Epoch  190
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Ru

Initializing cluster centers with louvain, resolution =  0.25000000000000006
Epoch  0
Epoch  10
delta_label  0.0032381513099793936 < tol  0.005
Reach tolerance threshold. Stopping training.
Total epoch: 19
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 86.15890619911647]
Run 2: l [0.01, 500.005], p [0.0, 21.868276596069336]
Run 3: l [0.01, 250.0075], p [0.0, 4.872209072113037]
Run 4: l [0.01, 125.00874999999999], p [0.0, 0.5367431640625]
Run 5: l [62.509375, 125.00874999999999], p [0.0004094839096069336, 0.5367431640625]
Run 6: l [93.7590625, 125.00874999999999], p [0.08291316032409668, 0.5367431640625]
Run 7: l [109.38390625, 125.00874999999999], p [0.25643062591552734, 0.5367431640625]
Run 8: l [117.196328125, 125.00874999999999], p [0.38443589210510254, 0.5367431640625]
Run 9: l [121.1025390625, 125.00874999999999], p [0.4577580690383911, 0.5367431640625]
recommended l =  123.05564453125
Start at res =  0.7 step =  0.

Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  10
Initializing cluster centers with louvain, resolution =  0.6
Epoch  0
Epoch  10
Res =  0.6 Num of clusters =  9
Res changed to 0.6
Initializing cluster centers with louvain, resolution =  0.5
Epoch  0
Epoch  10
Res =  0.5 Num of clusters =  7
Res changed to 0.5
Initializing cluster centers with louvain, resolution =  0.4
Epoch  0
Epoch  10
Res =  0.4 Num of clusters =  7
Res changed to 0.4
Initializing cluster centers with louvain, resolution =  0.30000000000000004
Epoch  0
Epoch  10
Res =  0.30000000000000004 Num of clusters =  5
Res changed to 0.30000000000000004
Initializing cluster centers with louvain, resolution =  0.20000000000000004
Epoch  0
Epoch  10
Res =  0.20000000000000004 Num of clusters =  4
recommended res =  0.20000000000000004
Initializing cluster centers with louvain, resolution =  0.20000000000000004
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch 